In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.14.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.71.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.6.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.21.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.38.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.9.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.42.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.32.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.20.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.27.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.30.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/mouth.51.png
/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set/mouth/m

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ustawienie ścieżki do katalogu z danymi
data_dir = '/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/training_set'

# Definicja generatora obrazów
datagen = ImageDataGenerator(
    rescale=1.0/255,           # Przeskalowanie wartości pikseli do zakresu [0, 1]
    validation_split=0.2       # Procent danych przeznaczonych na zbiór walidacyjny
)

# Wczytanie danych treningowych
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),    # Rozmiar docelowy obrazów
    batch_size=32,             # Rozmiar wsadu
    class_mode='categorical',  # Rodzaj klasyfikacji (binary lub categorical)
    subset='training'          # Ustawienie dla danych treningowych
)


# Wczytanie danych walidacyjnych
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'        # Ustawienie dla danych walidacyjnych
)

Found 320 images belonging to 2 classes.
Found 79 images belonging to 2 classes.


In [3]:
# Wczytanie danych treningowych
test_generator = datagen.flow_from_directory(
    '/kaggle/input/eye-vs-mouth-olivetti-faces-dataset/test_set',
    target_size=(224, 224),    # Rozmiar docelowy obrazów
    batch_size=32,             # Rozmiar wsadu
    class_mode='categorical',  # Rodzaj klasyfikacji (binary lub categorical)
    subset='training'          # Ustawienie dla danych treningowych
)


Found 87 images belonging to 2 classes.


In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D


num_epochs = 1
num_classes = 2

# Loading the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing all layers of the ResNet50
for layer in base_model.layers:
    layer.trainable = False

# Adding new classification layers
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compiling the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


model.fit(
    train_generator,
    epochs=1,                 # Liczba epok treningu
    validation_data=validation_generator
)

10/10 [==============================] - 54s 5s/step - loss: 7.1842 - accuracy: 0.7625 - val_loss: 0.5167 - val_accuracy: 0.7975


In [5]:
y_pred = model.predict(test_generator)

3/3 [==============================] - 3s 890ms/step


In [6]:
accuracy = model.evaluate(test_generator)


3/3 [==============================] - 3s 892ms/step - loss: 0.5625 - accuracy: 0.7356
